In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("dataset.csv")

In [ ]:
data.shape

In [ ]:
x_data = data['pixels']
y_data = data['emotion']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="darkgrid")
ax = sns.countplot(x="emotion", data=data)

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.show()


In [ ]:
%pip install imbalanced-learn

In [ ]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler

# Assuming x_data and y_data are pandas Series
x_data_array = np.array(x_data).reshape(-1, 1)
y_data_array = np.array(y_data)

# Create RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto')

# Resample the data
x_resampled, y_resampled = oversampler.fit_resample(x_data_array, y_data_array)

print(x_resampled.shape, y_resampled.shape)


In [ ]:
y_data.value_counts()

In [ ]:
x_data = pd.Series(x_data.values.flatten())
x_data

In [ ]:
x_data = np.array(list(map(str.split, x_data)), np.float32)
x_data/=255

In [ ]:
y_data = np.array(y_data)
y_data = y_data.reshape(y_data.shape[0], 1)
y_data.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1, random_state = 45)
print(x_train.shape," ",y_train.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# ***KNN CLASSIFIER***

In [ ]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train,y_train)

In [ ]:
knn.score(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred=knn.predict(x_test)
cm=confusion_matrix(y_test,y_pred)

In [ ]:
cm

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(7,5))
sns.heatmap(cm,annot=True)
plt.xlabel("Predicted")
plt.ylabel("Truth")

In [ ]:
from joblib import dump, load
dump(knn, 'knn.joblib')

# ***RANDOM FOREST***

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred=model.predict(x_test)
cm=confusion_matrix(y_test,y_pred)

In [ ]:
cm

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(7,5))
sns.heatmap(cm,annot=True)
plt.xlabel("Predicted")
plt.ylabel("Truth")

In [ ]:
from joblib import dump, load
dump(model, 'random_forest_model.joblib')

# GRADIO.APP

In [ ]:
%pip install gradio


RANDOM FOREST MODEL

In [ ]:
%pip install --upgrade typing-extensions

In [ ]:
%pip install -U gradio


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
from joblib import load

# Load the trained model
model = load('random_forest_model.joblib')

emotion_labels = {
    0: "Angry",
    1: "Fear",
    2: "Sad",
    3: "Neutral",
    4: "Happy",
    5: "Other"
}

def preprocess_image(image):
    # Convert the image to grayscale
    gray_image = Image.fromarray(image).convert('L')
    # Resize the image to match the model's expected input size
    resized_image = gray_image.resize((48,48))
    # Flatten the image to a 1D array and normalize
    processed_image = np.array(resized_image).flatten().reshape(1, -1) / 255.0
    return processed_image

def predict_emotion(image):
    try:
        processed_image = preprocess_image(image)
        prediction = model.predict(processed_image)[0]
        emotion = emotion_labels[prediction]
        return emotion
    except Exception as e:
        print(f"Error predicting emotion: {str(e)}")
        return "Other"

iface = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(),
    outputs="text",
    title="Emotion Detection",
    description="Tải ảnh lên để dự đoán cảm xúc."
)

iface.launch()


KNN CLASSIFIER

In [ ]:
%pip uninstall gradio
%pip install gradio


In [ ]:
import gradio as gr

print(gr.__version__)


In [ ]:
%pip install -U gradio numpy pillow scikit-learn


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
from joblib import load

# Load the trained model
model = load('knn.joblib')

emotion_labels = {
    0: "Angry",
    1: "Fear",
    2: "Sad",
    3: "Neutral",
    4: "Happy",
    5: "Other"
}

def preprocess_image(image):
    # Convert the image to grayscale
    gray_image = Image.fromarray(image).convert('L')
    # Resize the image to match the model's expected input size
    resized_image = gray_image.resize((48, 48))
    # Flatten the image to a 1D array and normalize
    processed_image = np.array(resized_image).flatten().reshape(1, -1) / 255.0
    return processed_image

def predict_emotion(image):
    try:
        processed_image = preprocess_image(image)
        prediction = model.predict(processed_image)[0]
        emotion = emotion_labels[prediction]
        return emotion
    except Exception as e:
        print(f"Error predicting emotion: {str(e)}")
        return "Other"

iface = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(),
    outputs="text",
    title="Emotion Detection",
    description="Upload a grayscale image, convert it to black and white, and predict the emotion."
)

iface.launch()
